In [11]:
gene_id <- "./test/dmg_Single_4"
iqtree_path <- paste0(gene_id, ".iqtree")
# open iqtree file as text
iqtree_text <- readLines(iqtree_path)
# abstract iqtree info
iqtree_summarise <- list()

In [12]:
# the best model and its criteria
line_best_model <- grep("Best-fit model according to", iqtree_text)
iqtree_summarise$model_crireia <- sub(".*to\\s+(.*):.*", "\\1", iqtree_text[line_best_model])
iqtree_summarise$best_model <- sub(".*:\\s*(.*)", "\\1", iqtree_text[line_best_model])
line_outgroup <- grep("Tree is UNROOTED", iqtree_text)
iqtree_summarise$outgroup <- sub(".*'\\s*(.*)'.*", "\\1", iqtree_text[line_outgroup])
rm(line_best_model, line_outgroup)


In [13]:
# Find the line numbers for the substitution model and likelihood tree
line_sub <- grep("SUBSTITUTION PROCESS", iqtree_text)
line_likelihood <- grep("MAXIMUM LIKELIHOOD TREE", iqtree_text)

# Extract the substitution model information
model_info <- iqtree_text[(line_sub+3):(line_likelihood-1)]
model_substitution <- sub(".*: (.*)", "\\1", model_info[1])
model_components <- unlist(strsplit(model_substitution, "\\+"))

# Check if the model includes a rate heterogeneity mixture
if (grepl("(m|M)ixture", model_info[1])){
    # Find the line numbers for the rate heterogeneity and site proportion and rates sections
    line_RHAS <-  grep("Model of rate heterogeneity:", model_info)

    # Extract the rate heterogeneity and site proportion and rates information
    iqtree_summarise$model_info <- read.table(text = model_info[3:(line_RHAS-1)], header = TRUE)
    if (grepl("R", model_components[length(model_components)])){
        line_table <- grep("Category", model_info)
        block_SPR <- model_info[line_table:(length(model_info) - 1)]
        iqtree_summarise$RHAS <- read.table(text = block_SPR, header = TRUE)
        rm(line_table, block_SPR)
    } else if (grepl("G", model_components[length(model_components)])){
        line_table <- grep("Category", model_info)
        block_SPR <- model_info[line_table:(length(model_info) - 2)]
        iqtree_summarise$RHAS <- read.table(text = block_SPR, header = TRUE)
        iqtree_summarise$G_alpha <-  sub(".*: ([0-9.]+)", "\\1", model_info[line_table - 2])
        rm(line_table, block_SPR)
    } else { 
        iqtree_summarise$RHAS <- NULL
    }
    
} else {
    # Find the line numbers for the rate parameter R, state frequencies, rate matrix Q, rate heterogeneity, and site proportion and rates sections
    line_R <- grep("Rate parameter R", model_info)
    line_F <- grep("State frequencies", model_info)
    line_Q <- grep("Rate matrix Q", model_info)
    line_RHAS <- grep("Model of rate heterogeneity:", model_info)
    
    # Extract the rate parameter R, state frequencies, and rate matrix Q information
    block_R <- model_info[(line_R+2):(line_F-2)]
    block_Q  <- model_info[(line_Q+2):(line_RHAS-1)]
    
    iqtree_summarise$R <- as.numeric(sub(".*: ([0-9.]+)", "\\1", block_R))
    names(iqtree_summarise$R) <- sub("(.*): [0-9.]+", "\\1", block_R)

    if (grepl("equal frequencies", model_info[line_F])){
        iqtree_summarise$F <- rep(1/4, 4)
    } else {
        block_F  <- model_info[(line_F+2):(line_Q-2)]
        iqtree_summarise$F <- as.numeric(sub(".*= ([0-9.]+)", "\\1", block_F))
        rm(block_F)
    }
    names(iqtree_summarise$F) <- c("A", "C", "G", "T")

    iqtree_summarise$Q <- read.table(text = block_Q, header = FALSE, row.names = 1)
    colnames(iqtree_summarise$Q) <- rownames(iqtree_summarise$Q)
    
    # Create a parameter string for the model
    parameter <- paste0(model_components[1], "{", paste(as.character(iqtree_summarise$R), collapse = ","),  "}+",
                        model_components[2], "{", paste(as.character(iqtree_summarise$F), collapse = ","),  "}")
    
    # Store the model information in a data frame
    iqtree_summarise$model_info <- data.frame("No" = 1,
                                  "Component" = model_components[1],
                                  "Rate" = 1.0000,
                                  "Weight" = 1.0000,
                                  "Parameters" = parameter)


    # Extract the rate heterogeneity and site proportion and rates information
    if (grepl("R", model_components[length(model_components)])){
        line_table <- grep("Category", model_info)
        block_SPR <- model_info[line_table:(length(model_info) - 1)]
        iqtree_summarise$RHAS <- read.table(text = block_SPR, header = TRUE)
        rm(line_table, block_SPR)
    } else if (grepl("G", model_components[length(model_components)])){
        line_table <- grep("Category", model_info)
        block_SPR <- model_info[line_table:(length(model_info) - 2)]
        iqtree_summarise$RHAS <- read.table(text = block_SPR, header = TRUE)
        iqtree_summarise$G_alpha <-  sub(".*: ([0-9.]+)", "\\1", model_info[line_table - 2])
        rm(line_table, block_SPR)
    } else { 
        iqtree_summarise$RHAS <- NULL
    }

    # Remove unnecessary variables
    rm(line_sub, line_R, line_F, line_Q, line_RHAS)
    rm(block_R, block_Q)
    
}

In [14]:
# tree length
line_length <- grep("Total tree length", iqtree_text)
iqtree_summarise$total_tree_length <- sub(".*: ([0-9.]+)", "\\1", iqtree_text[line_length])
iqtree_summarise$internal_tree_length <- sub(".*: ([0-9.]+).*", "\\1", iqtree_text[line_length + 1])
rm(line_length)


In [15]:
# statistics

iqtree_summarise$log_likelihood <- sub(".*: ([0-9.-]+) .*", "\\1", iqtree_text[line_likelihood + 3])
iqtree_summarise$log_likelihood_sd <- sub(".*: ([0-9.-]+) \\(s.e. ([0-9.]+)\\)", "\\2", iqtree_text[line_likelihood + 3])
iqtree_summarise$log_likelihood_Unconstrained <- sub(".*: ([0-9.-]+)", "\\1", iqtree_text[line_likelihood + 4])
iqtree_summarise$num_free_params <- sub(".*: ([0-9]+)", "\\1", iqtree_text[line_likelihood + 5])
iqtree_summarise$AIC  <- sub(".*: ([0-9.-]+)", "\\1", iqtree_text[line_likelihood + 6])
iqtree_summarise$AICc <- sub(".*: ([0-9.-]+)", "\\1", iqtree_text[line_likelihood + 7])
iqtree_summarise$BIC  <- sub(".*: ([0-9.-]+)", "\\1", iqtree_text[line_likelihood + 8])
# iqtree_summarise$num_params <- sub(".*: ([0-9]+)", "\\1", iqtree_text[line_likelihood + 11])
# iqtree_summarise$samp_size <- sub(".*: ([0-9]+)", "\\1", iqtree_text[line_likelihood + 12])
rm(line_likelihood)


In [16]:
names(iqtree_summarise)

[1] "model_crireia"                "best_model"                  
 [3] "outgroup"                     "R"                           
 [5] "F"                            "Q"                           
 [7] "model_info"                   "RHAS"                        
 [9] "G_alpha"                      "total_tree_length"           
[11] "internal_tree_length"         "log_likelihood"              
[13] "log_likelihood_sd"            "log_likelihood_Unconstrained"
[15] "num_free_params"              "AIC"                         
[17] "AICc"                         "BIC"

In [17]:
paras_common <- c('model_crireia','best_model','model_info','RHAS',
                  'total_tree_length','internal_tree_length','log_likelihood',
                  'log_likelihood_sd','log_likelihood_Unconstrained','num_free_params',
                  'AIC','AICc','BIC')
paras_onemodel <- c(paras_common, "R", "F", "Q")